# Programming Assignment: 


1. На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
2. Реализуйте два алгоритма рекомендаций:

    сортировка просмотренных id по популярности (частота появления в просмотренных),
    сортировка просмотренных id по покупаемости (частота появления в покупках).

3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие?           Обратите внимание на различие качества на обучающей и тестовой           выборке в случае рекомендаций по частотам покупки.



In [1]:
import numpy as np
import pandas as pd
from scipy import linalg
import matplotlib.pylab as plt

import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

%matplotlib inline


**Входные данные**

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

**Важно:**

* Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
* Если товар не встречался в обучающей выборке, его популярность равна 0.  
* Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
* Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

In [2]:
f = open("test.txt", "r")
test = f.read().splitlines() 
test[:5]

['6,7,8;', '13,14,15;', '22,23;', '28,29,30,31,32,33;', '40,41;']

In [3]:
f = open("train.txt", "r")
train = f.read().splitlines() 
train[:5]

['0,1,2,3,4,5;',
 '9,10,11,9,11,12,9,11;',
 '16,17,18,19,20,21;',
 '24,25,26,27,24;',
 '34,35,36,34,37,35,36,37,38,39,38,39;']

# Task №1
На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)


In [4]:
def data_preparation(path):
    f = open(path, "r")
    text = f.read().splitlines() 
    data = []

    for line in text:
        seen, bought = line.split(';')
        if bought != '':
            data.append([seen.split(","), bought.split(",")])
        else: 
            data.append([seen.split(","), []])
    return data

In [5]:
test = data_preparation("test.txt")
train = data_preparation("train.txt")

In [6]:
def unique_id(data, n):
    id_unique = {}
    for i in range(len(data)):
        for id_value in (data[:][i])[n]:
            if id_value not in id_unique:
                id_unique.update({id_value: 0})
            else:
                id_unique.update({id_value: id_unique.get(id_value)+1})
    return id_unique

In [7]:
id_seen = unique_id(test, 0)
id_bought = unique_id(test, 1)

# Task №2
Реализуйте два алгоритма рекомендаций:

* сортировка просмотренных id по популярности (частота появления в просмотренных),
* сортировка просмотренных id по покупаемости (частота появления в покупках).



In [8]:
id_seen_sorted = sorted(id_seen.items(), key=lambda x: x[1], reverse=True)
id_bought_sorted = sorted(id_bought.items(), key=lambda x: x[1], reverse=True)

In [9]:
'''
for i in range(len(train)):
    for id_value in (train[:][i])[1]: 
        if id_value == []:
            continue
        else:   #что-то купили
            print(id_value)
'''

'\nfor i in range(len(train)):\n    for id_value in (train[:][i])[1]: \n        if id_value == []:\n            continue\n        else:   #что-то купили\n            print(id_value)\n'

2 Берем строчку, где что-то купили. Смотрим просмотры в ней. Ищем для каждого уникального id в просмотрах соответствие в каком-то словаре из п 1, смотря на просмотрах или покупках мы строим рекомендации. Сортируем по этим значениям. Если одинаковые, ставим первым тот, что первым появился в этой строке в просмотрах. Отбираем k1 первых чисел получившегося массива (k1 = минимум из k и количества уникальных просмотров данной строки), если он длиннее k1. Этот массив - наша рекомендация.

In [10]:
recommend_dict = {}

for i in range(len(train)):
    sess_bought = train[:][i][1]
    if sess_bought != []:   #что-то купили
        sess_seen = train[:][i][0] #Ищем для каждого уникального id в просмотрах
        
        sess_seen = set(sess_seen)
        for id_value in sess_seen: 
            if id_value in id_seen.keys():
                #print(id_value, id_seen.get(id_value))
            #if id_value in sess_seen:
                

SyntaxError: unexpected EOF while parsing (<ipython-input-10-9cdc2b89b106>, line 13)

# Task №3
Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие?           Обратите внимание на различие качества на обучающей и тестовой           выборке в случае рекомендаций по частотам покупки.

In [ ]:
def AverageRecall(n, arr):
    